In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src/')  
from scraper.session import login, BASE_URL
from scraper.fetch import get_court_links
# from scraper.parser import parse_cases
from scraper.court_scraper import CourtScraper



# getting court links doesnt need session as no log in... but it does want you to log in to view the deetts..
links = get_court_links()
# print(links)

# logging in
session = login()      


# parse cases using session data from login
# parse_cases(session, links)
for link in links:
    courtScraoer = CourtScraper(session, BASE_URL + link)
    courtScraoer.load_case_page()
    courtScraoer.get_case_list_soup()
    courtScraoer.extract_city_and_court_name()
    courtScraoer.get_cleaned_row_texts()






getting court links
username: analogue1@live.com, password: xtmpw463
Status Code: 302
Final URL: https://courtserve.net/
session cookies: <RequestsCookieJar[<Cookie SITELOKPW=cjV6cEpPQldGNDlRK3BHcWhiYXFNdjZscUZJQ0cwRytTcG5TdkVvPQ%3D%3D for .courtserve.net/>, <Cookie PHPSESSID=5e0f17230a5669a15041a509892563c43773c490 for courtserve.net/>]>
probs logged in
 extracted Aldershot & Farnham from the court name
1 hour
 extracted Barnet from the court name
2
    hours
 extracted Barnsley from the court name
1
    hour
30     minutes
6
    hours
 extracted Basildon from the court name


IndexError: list index out of range